In [49]:
from pathlib import Path
import pandas as pd

only_silences = True

AUDIO_PATH = Path("../data/raw/2024/unlabeled_soundscapes")
FILE_PATHS = list(AUDIO_PATH.glob("*.ogg"))
PREDICTIONS_OUTPUT_PATH = Path("../data/raw/2024/google-bvc-predictions.csv")
SAMPLE_RATE = 32000
OUTPUT_PATH = Path("../data/raw/2024google/train_audio/")
META_PATH = Path("../data/raw/2024google/train_metadata.csv")

In [50]:
#Load in preds

df_preds = pd.read_csv(PREDICTIONS_OUTPUT_PATH)
#Get the rows that 


In [69]:
from tqdm import tqdm
import librosa
import soundfile as sf

train_metadata = {"filename": [], "labels": [], "rating": [], "primary_label": []}


if only_silences:
    # Get the rows where the sum of the classes is <= 0.1
    sliced = df_preds[df_preds.iloc[:,:182].sum(axis=1) <= 0.004]
    print(len(sliced) / len(df_preds))
    sliced["file"] = sliced["row_id"].apply(lambda x: x.split("_")[0])
    
    # Group by the file and apply operations per group in loop
    for group in tqdm(sliced.groupby("file"), desc="Processing files"):
        file_name = group[0]


        curr_df = group[1]
        file = AUDIO_PATH /file_name
        
        segments = curr_df["row_id"].apply(lambda x: int(x.split("_")[1])).values
        # print(segments)
        
        #Load the file using librosa
        y, sr = librosa.load(file, sr=SAMPLE_RATE)
        #Replace .ogg with empty
        file_name = file_name.replace(".ogg", "")
        #Slice the audio file into 5 second chunks
        for segment in segments:
            start = segment * SAMPLE_RATE
            end = (segment + 5) * SAMPLE_RATE
            
            chunk = y[start:end]
            
            #Save the chunk to disk
            output_file = OUTPUT_PATH / f"{file_name}_{segment}.wav"
            sf.write(output_file, chunk, SAMPLE_RATE, format="WAV")
            

            #Append the metadata to the metadata dict
            train_metadata["filename"].append(f"{file_name}_{segment}.wav")
            train_metadata["labels"].append([])
            train_metadata["rating"].append(5)
            train_metadata["primary_label"].append("silent")
        
    #Save metatdata to disk
    metadata_df = pd.DataFrame(train_metadata)
    metadata_df.to_csv(META_PATH, index=False)
        
        

    


/tmp/ipykernel_2365675/1380949178.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced["file"] = sliced["row_id"].apply(lambda x: x.split("_")[0])


0.03788649511270696


Processing files: 100%|██████████| 1957/1957 [04:47<00:00,  6.81it/s]
